In [1]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
stocks = ["MSFT", "AAPL", "NVDA", "AMZN", "GOOG"]

In [3]:
def get_article(card):
  headline = card.find('h4','s-title').text
  source = card.find('span','s-source').text
  posted = card.find('span','s-time').text.replace('.','').strip()
  description = card.find('p','s-desc').text.strip()
  link = card.find('h4').find('a')['href']

  article = (headline,source,posted,description,link)
  return article

In [4]:
def get_the_news(search):

  template = 'https://news.search.yahoo.com/search?p={}'
  url = template.format(search)

  articles = []
  links = set()

  while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        cards = soup.find_all('div', 'NewsArticle')

        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)

        try:
            url = soup.find('a', 'next').get('href')
            sleep(2)

        except AttributeError:
            break

  df = pd.DataFrame(articles, columns=['Headline', 'Source', 'Posted', 'Description', 'Link'])

  return df

In [5]:
def convert_relative_time_to_date(relative_time):
    numerical_value = int(re.search(r'\d+', relative_time).group())

    if 'minute' in relative_time:
        return (datetime.now() - timedelta(minutes=numerical_value)).date()
    elif 'hour' in relative_time:
        return (datetime.now() - timedelta(hours=numerical_value)).date()
    elif 'day' in relative_time:
        return (datetime.now() - timedelta(days=numerical_value)).date()
    else:
        return None

In [6]:
for stock in stocks:
    news = get_the_news(stock)
    news['Posted'] = news['Posted'].apply(convert_relative_time_to_date)
    news.to_csv(stock + '.csv')
    print(stock + " saved!")

MSFT saved!
AAPL saved!
NVDA saved!
AMZN saved!
GOOG saved!
